In [15]:
import os
import numpy as np
import pandas as pd
from time import time
from typing import List, Iterable

if not os.path.isfile("train.csv") or not os.path.isfile("test.csv"):
    print("Downloading data...")
    train = pd.read_csv("https://www.csie.ntu.edu.tw/~htlin/course/ml20fall/hw6/hw6_train.dat", sep=r"\s", header=None)
    test = pd.read_csv("https://www.csie.ntu.edu.tw/~htlin/course/ml20fall/hw6/hw6_train.dat", sep=r"\s", header=None)

    train.to_csv("train.csv", index=False)
    test.to_csv("test.csv", index=False)
    print("Completed")
else:
    train = pd.read_csv("train.csv")
    test = pd.read_csv("test.csv")


class TreeNode:
    def __init__(self, threshold, feature, value=None):
        self.left = None
        self.right = None
        self.value = value
        self.feature = feature
        self.threshold = threshold


def tree_predict(tree: TreeNode, data: List[float]) -> float:
    while (not tree.value):
        if data[int(tree.feature)] >= tree.threshold:
            tree = tree.right
        else:
            tree = tree.left

    return tree.value


def learn_tree(data: pd.DataFrame) -> List[List]:
    if is_pure(data):
        return TreeNode(None, None, data["10"].unique()[0])
    else:
        before = time()
        featureNum, threshold = find_best_feature(data)
        after = time()
        print(f"{after - before:.2f}")
        rightBranch = data[(data[featureNum] >= threshold)]
        leftBranch = data[(data[featureNum] < threshold)]
        DecisionTreeClassifier = TreeNode(threshold, featureNum, None)
        DecisionTreeClassifier.right = learn_tree(rightBranch)
        DecisionTreeClassifier.left = learn_tree(leftBranch)

        return DecisionTreeClassifier


def is_pure(data: pd.Series) -> bool:
    return len(pd.unique(data["10"])) == 1


def find_best_feature(data: pd.DataFrame) -> List:
    features = data.drop(["10"], axis=1).columns
    purityList = [find_best_threshold(data, feature) for feature in features]
    _, threshold, featureNumber = sorted(purityList, key=lambda x: (x[0], x[1], int(x[2])), reverse=False)[0]

    return [str(featureNumber), threshold]


def find_best_threshold(data: pd.DataFrame, feature: str) -> List[float]:
    featureList = []
    for threshold in generate_threshold(data[feature]):
        rightBran = data["10"][(data[feature] >= threshold)]
        leftBran = data["10"][(data[feature] < threshold)]
        weight = rightBran.count() / (rightBran.count() + leftBran.count())
        impurity = weight * gini_coef(rightBran) + (1 - weight) * gini_coef(leftBran)
        featureList.append([impurity, threshold, feature])

    return sorted(featureList, key=lambda x: (x[0], x[1], x[2]), reverse=False)[0]


def generate_threshold(data: pd.Series) -> List[float]:
    tmp = sorted(data.to_list())
    thresholdList = []
    for i in range(len(tmp)):
        if i == (len(tmp) - 1):
            break
        thresholdList.append((tmp[i] + tmp[i + 1]) / 2)

    return list(set(thresholdList))


def gini_coef(data: pd.Series) -> float:
    probability = (data.value_counts() / len(data)).values
    return 1 - np.sum(np.square(probability))

In [2]:
%%time

tree = learn_tree(train)

16.69
15.35
1.77
0.29
0.24
1.47
1.38
0.44
0.42
0.05
0.22
0.07
12.64
12.88
9.61
9.06
9.45
10.70
7.49
0.16
7.80
0.15
7.67
0.23
0.10
6.36
0.15
0.03
6.66
0.10
6.46
6.42
0.72
0.66
0.04
0.62
0.20
0.11
0.06
5.54
5.67
5.54
1.57
1.69
1.72
1.58
1.52
0.33
0.31
0.11
0.05
4.70
4.52
0.11
4.46
0.47
0.02
0.56
0.22
0.59
0.14
0.37
0.35
0.11
0.03
0.27
0.03
0.43
0.42
0.03
0.87
0.49
0.23
0.18
0.16
0.24
0.27
0.02
0.39
0.03
1.12
1.07
0.53
0.31
0.19
0.16
0.12
Wall time: 3min 24s


In [16]:
tree_predict(tree, test.iloc[0].to_list()[:10])

1.0